In [ ]:
import _ParlayANNpy as pann
import wrapper as wp
import os
import numpy as np
import heapq
from tqdm import tqdm
import matplotlib.pyplot as plt

print(dir(pann))

FERN_DATA_DIR = "/ssd1/anndata/bigann/"
AWARE_DATA_DIR = "/ssd1/data/bigann/"
HARSHA_DATA_DIR = "/home/ben/Documents/data/yfcc100M/"

DATA_DIR = HARSHA_DATA_DIR

output_filename = os.path.join(DATA_DIR, "vamana_new")


In [ ]:

# wp.build_pynndescent_index("Euclidian", "uint8", DATA_DIR + "base.1B.u8bin.crop_nb_1000000", DATA_DIR + "outputs/pynn", 40, 10, 100, 1.2, .05)
# wp.build_vamana_index("Euclidian", "uint8", os.path.join(DATA_DIR, "base.10M.u8bin.crop_nb_10000000"), output_filename, 64, 200, 1.2, True)

# print("Index built")


In [ ]:

Index = wp.load_index("Euclidian", "uint8", os.path.join(DATA_DIR, "base.10M.u8bin.crop_nb_10000000"), output_filename, 10000000, 192, False)
neighbors, distances = Index.batch_search_from_string(DATA_DIR + "query.public.100K.u8bin", 100000, 10, 100)

Index.check_recall(DATA_DIR + "GT.public.ibin", neighbors, 10)


In [ ]:
out_neighbors, lengths = Index.edges_and_lengths()

print(out_neighbors[0], lengths[0])
print(out_neighbors.shape, lengths.shape)

In [ ]:
# how many points have an edge to the first point?
print(sum(out_neighbors[0] == 0), sum())

In [ ]:
# dijkstra's to compute the walk distance from the first point to all other points
walk_distances = np.full(10000000, np.iinfo(np.uint32).max, dtype=np.uint32)
walk_distances[0] = 0
visited = np.zeros(10000000, dtype=bool)

pbar = tqdm(total=10000000)

priority_queue = []
heapq.heappush(priority_queue, (0, 0))  # (distance, node)

pbar = tqdm(total=10000000)

while priority_queue:
    current_distance, current_node = heapq.heappop(priority_queue)
    
    if visited[current_node]:
        continue
    
    # Mark as visited
    visited[current_node] = True
    pbar.update(1)
    
    # Process each neighbor
    for neighbor in out_neighbors[current_node]:
        if neighbor < 10000000 and not visited[neighbor]:
            new_distance = current_distance + 1  # assuming each edge has weight 1
            if new_distance < walk_distances[neighbor]:
                walk_distances[neighbor] = new_distance
                heapq.heappush(priority_queue, (new_distance, neighbor))
    
pbar.close()


# write walk distances
print("Writing walk distances")
np.save(DATA_DIR + "walk_distances.npy", walk_distances)
print("Done")

In [ ]:
# read walk distances
print("Reading walk distances")
walk_distances = np.load(DATA_DIR + "walk_distances.npy")
print("Done")
connected_walk_distances = walk_distances[walk_distances < 10000000]

print(len(connected_walk_distances), len(connected_walk_distances) / 10000000, np.max(connected_walk_distances), np.mean(connected_walk_distances), np.median(connected_walk_distances))

In [ ]:
all_lengths = lengths
flattened_lengths = lengths.flatten()[out_neighbors.flatten() < 10000000]
sqrt_lengths = np.sqrt(flattened_lengths)
# computing the lengths of the edges proportional to the first edge
proportional_lengths = (lengths / lengths[0]).flatten()[out_neighbors.flatten() < 10000000]

In [ ]:

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].hist(sqrt_lengths, bins=100)
axs[1].hist(proportional_lengths, bins=100)
plt.show()


In [ ]:
# %%
plt.hist(np.log(sqrt_lengths), bins=100)
plt.show()


In [ ]:
# %%
plt.hist(walk_distances[walk_distances < 10000000], bins=np.max(walk_distances[walk_distances < 10000000]))
plt.show()


In [ ]:
# %%
# plot the distribution of the lengths of the edges at different dijkstra distances
# the histogram is normalized by the number of edges at each distance
fig, axs = plt.subplots(1, 1, figsize=(10, 5))

